# Chainerチュートリアル　Function
まずはコマンド・プロンプトないしそれ相当のもので"pip install chainer"

In [1]:
import chainer
import numpy as np
from chainer import cuda, Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions

In [2]:
x_data = np.array([5], dtype=np.float32)
x = Variable(x_data)

Variable型はChaunier独自の型で変数を格納できるみたい

In [3]:
print(x_data)
print(x)

[ 5.]
<var@2560fcf6d30>


ここからはVariableの性質を見ていく

In [4]:
y = x**2 - 2*x + 1

In [5]:
y

中身はそのままだと見れない

In [6]:
print(y)

<var@2560fcf6c18>


In [7]:
y.data

array([ 16.], dtype=float32)

In [8]:
print(y.data)

[ 16.]


これで見れる。値はdataに格納される。ChainerのVariable型は値の他に履歴や便利な機能を提供するために直接的には値が参照できない。その便利な機能の一つが以下のbackward()

In [9]:
print(y.backward())

None


別段backward()そのものは何も返さないが…

In [10]:
print(x.grad)

[ 8.]


In [11]:
x.grad

array([ 8.], dtype=float32)

x.gradに変化がある。これはつまりyのxに関する勾配が格納されている。今回ならdy/dx = 2x -2, x = 5なので 8で確かに正しい。

In [12]:
y.backward()

In [13]:
print(x.grad)

[ 16.]


もう一度backwardをすると値が累積するみたいだ。これはニューラルネットで実際にバックプロパゲーションするときにバッチ処理（たくさんのデータをまとめて処理すること）するのに便利だからだろう。自信は…無い。

In [14]:
x.grad = np.array([0], dtype=np.float32)
print(x.grad)
y.backward()
print(x.grad)

[ 0.]
[ 8.]


一応値はリセットできるみたいだ。ただし注意が必要で、ここで初めて気づいたがどうやらChainerは基本の型をnumpyのnp.float32型としているらしく代入する値をこれ以外にするとエラーを吐く。

次は中間に変数がある場合について見てみる。

In [15]:
x_data = np.array([5.], dtype=np.float32)
x = Variable(x_data)
z = 2*x
y = x**2 - z + 1
print("retain_grad = True")
y.backward(retain_grad=True)
print("zgrad:",z.grad, ", z:", z.data)
print("xgrad:",x.grad, ", x:", x.data)
print("retain_grad = False")
x = Variable(x_data)
z = 2*x
y = x**2 - z + 1
y.backward(retain_grad=False)
print("zgrad:",z.grad, ", z:", z.data)
print("xgrad:",x.grad, ", x:", x.data)

retain_grad = True
zgrad: [-1.] , z: [ 10.]
xgrad: [ 8.] , x: [ 5.]
retain_grad = False
zgrad: None , z: [ 10.]
xgrad: [ 8.] , x: [ 5.]


retain_grad をTrueにするとｚのgradにも値が格納される。あまり使うことは無いらしい。また、retain_gradの真偽に関わらず、ｘのgradはｚを展開した式の形になる。

In [34]:
x.data

array([ 5.], dtype=float32)

ChainerのFunctionモジュールの関数はVariable型を引数にとる物が多いらしく、これらを組み合わせることで複雑なバックプロパゲーション（backward()のこと）の計算ができるとのこと。

# Links

ニューラルネットでは素子と素子とを「繋げる」ということがとても大切になってくる。もちろんChainerではその動作が簡単に書けるのだ。

In [16]:
f = L.Linear(3, 2)

L.linear と　L.Linear どっちもあるって罠だな。  
ちなみにlinearの方は関数ですらない

これで f(x) = Wx + b (勿論Wは行列、bはベクトルだぞ)　の式を作ることができた。与えた引数によってWは　2* 3,　 b は2の長さになっているぞ

In [18]:
print("W:",f.W.data)
print("b:",f.b.data)

W: [[ 0.27023029 -0.81562287  0.10888684]
 [ 0.62706101  0.85545802  0.12873293]]
b: [ 0.  0.]


Wはランダムな値、bは０で初期化されている

In [20]:
f.W = Variable(np.array([[1,0,0],[0,1,0]], dtype=np.float32))
f.b = Variable(np.array([1,1], dtype=np.float32))

In [21]:
print("W:",f.W.data)
print("b:",f.b.data)

W: [[ 1.  0.  0.]
 [ 0.  1.  0.]]
b: [ 1.  1.]


実験のために値をわかりやすく指定した。

In [23]:
x = Variable(np.array([[1, 2, 3], [4, 5, 6]], dtype=np.float32))
y = f(x)
y.data

array([[ 2.,  3.],
       [ 5.,  6.]], dtype=float32)

{(1,0,0),(0,1,0)}(1,2,3) + (1,1) = (2, 3)  
{(1,0,0),(0,1,0)}(4,5,6) + (1,1) = (5, 6)  
まぁ、縦と横が逆な気がするけど良いんじゃないかな？  
これはdata[i]でx[i]に対する解が見えるようにするためだろう。